<h1 align="center"><img align="center" src="https://geoparse.io/graphics/geoparse_logo.png" alt="GeoParse Logo" width="200"/></h1>
<h1 align="center">GeoParse</h1>
<h3 align="center">All About Points <img src="https://geoparse.io/graphics/point.png" width="10"/> Lines <img src="https://geoparse.io/graphics/line.png" width="40"/> and Polygons <img src="https://geoparse.io/graphics/polygon.png" width="30"/></h3>


#### [HTML](http://geoparse.io/tutorials/karta.html) 
***

# Impute Postcodes

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/geoparse/geoparse/main?labpath=tutorials%2F00_visualization.ipynb)

This notebook demonstrates how to display:

* Points, lines, and polygons.
* Their heatmaps and clusters.
* Their coverage areas using geospatial cells such as H3, S2, and Geohash.
* Geospatial cells on a map based on their indexes.
* OSM roads and buildings using their IDs.
***

In [ ]:
import warnings

import pandas as pd
import geopandas as gpd

from geoparse.geoparse import plp, SnabbKarta

pd.set_option("display.max_columns", None)
warnings.filterwarnings("ignore")

# Postcode

In [ ]:
%%time
gdf = gpd.read_parquet('../data/os-codepoint-open/codepo_gb.parquet')
gdf.head()

In [ ]:
SnabbKarta.plp(gdf, point_color='admin_district_code')

In [ ]:
len(gdf)

In [ ]:
gdf.postcode.nunique()

In [ ]:
gdf.geometry.nunique()

In [ ]:
gdf.geometry.value_counts()

In [ ]:
gdf.loc[gdf.country_code.str.startswith('S', na=False), 'country_code'] = 'Scotland'
gdf.loc[gdf.country_code.str.startswith('E', na=False), 'country_code'] = 'England'
gdf.loc[gdf.country_code.str.startswith('W', na=False), 'country_code'] = 'Wales'
gdf.head()

In [ ]:
gdf.country_code.value_counts()

In [ ]:
ddf = pd.read_excel('../data/os-codepoint-open/Codelist.xlsx', sheet_name='CTY', header=None)
ddf.head()

In [ ]:
area_dict = {}
for sheet_name in ['CTY', 'DIS', 'DIW', 'LBO', 'LBW', 'MTD', 'MTW', 'UTA', 'UTE', 'UTW']:
    df = pd.read_excel('../../open-data/data/os-codepoint-open/Codelist.xlsx', sheet_name=sheet_name, header=None)
    for _, row in df.iterrows():
        area_dict[row[1]] = row[0]

In [ ]:
gdf['admin_district_code'] = gdf.admin_district_code.apply(lambda x: area_dict[x] if x in area_dict else x)
gdf['admin_ward_code'] = gdf.admin_ward_code.apply(lambda x: area_dict[x] if x in area_dict else x)
gdf['country_code'] = gdf.country_code.apply(lambda x: area_dict[x] if x in area_dict else x)
gdf.head()

In [ ]:
gdf.isnull().sum()

In [ ]:
# Split into known and missing district codes
kdf = gdf[gdf["admin_district_code"].notna()]     # known df
mdf = gdf[gdf["admin_district_code"].isna()]  # missing df
len(kdf), len(mdf)

In [ ]:
ndf = gpd.sjoin_nearest(   # nearest df
    mdf[['postcode', 'geometry']], 
    kdf[["admin_district_code", 'admin_ward_code', "geometry"]], 
    how="left", 
#    max_distance=None,   # optional: limit to a search radius
 #   distance_col="dist"
)
ndf

In [ ]:
ndf = ndf.drop(columns='index_right').drop_duplicates()
len(ndf)

In [ ]:
gdf.isnull().sum()

In [ ]:
gdf.loc[gdf.admin_district_code.isnull(), "admin_district_code"] = ndf["admin_district_code"]
gdf.loc[gdf.admin_ward_code.isnull(), "admin_ward_code"] = ndf["admin_ward_code"]

In [ ]:
gdf.isnull().sum()

In [ ]:
gdf.to_parquet('../data/os-codepoint-open/codepo_gb_imputed.parquet', index=False)

In [ ]:
df = pd.read_csv("https://geoparse.io/tutorials/data/fatal_crash_great_britain_2023.csv")
df.head()

In [ ]:
SnabbKarta.plp(df)

In [ ]:
SnabbKarta.plp(df, geom_col=['longitude', 'latitude'])

In [ ]:
%%time
gdf = gpd.read_parquet('../../open-data/data/os-open-usrn/osopenusrn_202509.parquet')#.to_crs('epsg:4326')
len(gdf)

In [ ]:
gdf.head()

In [ ]:
SnabbKarta.plp(gdf.head(), centroid=True)

In [ ]:
%%time
rdf = gpd.read_parquet('../../open-data/data/os-open-roads/road_link.parquet')#.to_crs('epsg:4326')
len(rdf)

In [ ]:
rdf.head()

In [ ]:
%%time
gdf = pd.read_csv('../../open-data/data/dft-road-safety/dft-road-casualty-statistics-casualty-1979-latest-published-year.csv.gz')#.to_crs('epsg:4326')
len(gdf)

In [ ]:
%%time
cdf = pd.read_csv('../../open-data/data/dft-road-safety/dft-road-casualty-statistics-casualty-1979-latest-published-year.csv')#.to_crs('epsg:4326')
len(cdf)

In [ ]:
pdf

In [ ]:
pdf.lsoa_of_casualty.value_counts()

In [ ]:
pdf.casualty_distance_banding[1]

In [ ]:
type(gdf.lsoa_of_casualty[11845977])

In [ ]:
gdf.info()

In [ ]:
pdf.info()

In [ ]:
df.head()

In [ ]:
df[df.local_authority_name=='Camden'].count_date.nunique()

In [ ]:
SnabbKarta.plp(df[df.local_authority_name=='Camden'])

In [ ]:
df = gpd.read_file('/Users/abbas/Downloads/dft/dft_traffic_counts_aadf.csv')

In [ ]:
df.year.value_counts()

In [ ]:

df.head()

In [ ]:
SnabbKarta.plp(df[df.year=='2024'], point_radius='pedal_cycles')

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
gdf = gpd.read_parquet('../../open-data/data/os-open-roads/Data/road_link.parquet')
len(gdf)

In [ ]:
gdf.head()

In [ ]:
gdf.form_of_way.value_counts(dropna=False)

In [ ]:
gdf.road_function.value_counts(dropna=False)

In [ ]:
SnabbKarta.plp(gdf[gdf.road_function=='B Road'])

In [ ]:
SnabbKarta.plp(gdf.sample(100000), 
               #line_color='street_type'
              )

In [ ]:
SnabbKarta.plp([df, dict1, dict2], point_radius=1000)

In [ ]:
pdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.longitude, df.latitude),
    crs="EPSG:4326"  # WGS 84
)
del pdf['latitude']
del pdf['longitude']
pdf.head()

In [ ]:
SnabbKarta.plp(pdf)

In [ ]:
df = pd.read_csv("../../sandbox/data/1m_random_2023-12.csv.gz", header=None)
df.columns = ['vin', 'ts', 'lat', 'lon', 'x', 'y', 'z', 'speed', 'bearing', 'isGPS', 'u', 'company']
pdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.lon, df.lat),
    crs="EPSG:4326"  # WGS 84
)
del pdf['lat']
del pdf['lon']
pdf.head()

In [ ]:
len(pdf)

Available tile layers are:
* **OSM:** A versatile map powered by OpenStreetMap, displaying roads, buildings, and points of interest.
* **Satellite:** Displays high-resolution satellite imagery for real-world context and detailed analysis.
* **Outdoors:** Designed for outdoor activities, featuring trails, elevation contours, and natural landmarks.
* **Dark:** A high-contrast, dark-themed map ideal for vibrant overlays and nighttime visualization.
* **Light:** A clean, minimalistic basemap that highlights overlaid data.

You can switch between different tile layers using the options in the top right corner of the map.

In [ ]:
SnabbKarta.plp(df)

In [ ]:
SnabbKarta.plp(df, h3_res=4)

In [ ]:
from functools import partial
from typing import Sequence

import ipywidgets as widgets
import traitlets

from lonboard import Map
from lonboard.basemap import CartoBasemap
from lonboard.models import ViewState

In [ ]:
## Create postitron map focused on the arch
positron_map = Map(
    layers=layer,
    basemap_style=CartoBasemap.Positron,
    view_state={
        "longitude": -90.1849,
        "latitude": 38.6245,
        "zoom": 16,
        "pitch": 30,
        "bearing": 0,
    },
    _height=800,
)

positron_map

In [ ]:
## Create postitron map focused on the arch
positron_map = Map(
    layers=[],
    basemap_style=CartoBasemap.Positron,
    view_state={
        "longitude": -90.1849,
        "latitude": 38.6245,
        "zoom": 16,
        "pitch": 0,
        "bearing": 0,
    },
    layout=widgets.Layout(flex="1"),
)

## Create postitron map focused on the lady liberty
darkmatter_map = Map(
    layers=[],
    basemap_style=CartoBasemap.DarkMatter,
    view_state={
        "longitude": -74.04454,
        "latitude": 40.6892,
        "zoom": 16,
        "pitch": 0,
        "bearing": 0,
    },
    layout=widgets.Layout(flex="1"),
)

maps_box = widgets.HBox([positron_map, darkmatter_map])
maps_box

In [ ]:
def sync_positron_to_darkmatter(event: traitlets.utils.bunch.Bunch) -> None:
    if isinstance(event.get("new"), ViewState):
        darkmatter_map.view_state = positron_map.view_state


positron_map.observe(sync_positron_to_darkmatter)


def sync_darkmatter_to_positron(event: traitlets.utils.bunch.Bunch) -> None:
    if isinstance(event.get("new"), ViewState):
        positron_map.view_state = darkmatter_map.view_state


darkmatter_map.observe(sync_darkmatter_to_positron)

The `plp` function displays points in blue by default. However, you can change the point color using the `point_color` argument. 

In [ ]:
plp(df, point_color="purple")

In [ ]:
SnabbKarta.plp(df, point_color="purple")

For a custom color, use an `RGB` hex code like "#cc5500" for burnt orange.

In [ ]:
plp(df, point_color="#cc5500")

The `plp` function can group points by color based on their values, meaning points with the same value share the same color.
It assigns colors consistently by mapping input column values to a predefined color palette, ensuring a clear and structured visualization.

In [ ]:
plp(df, point_color="number_of_casualties")

To enhance data interpretation, `plp` allows tooltips that display attribute values when hovering over a point.
The `point_popup` parameter is a dictionary where keys define tooltip labels, and values correspond to column names in the DataFrame.

In [ ]:
plp(df, point_color="number_of_casualties", point_popup={"Date": "date", "Casualties": "number_of_casualties"})

By passing the `heatmap` argument as `True`, `plp` creates a heatmap layer for the points. 
For better visualization switch to `Dark` mode from the Layer Control menu in the top-right corner.

In [ ]:
plp(df, heatmap=True)

Setting `heatmap=True` disables the display of points. To show points alongside the heatmap layer, set `heatmap_only=False`.

In [ ]:
plp(df, heatmap=True, heatmap_only=False)

`plp` groups points together based on their proximity by setting the `cluster` argument to `True`.

In [ ]:
plp(df, cluster=True)

We can also display both the `heatmap` and `cluster` simultaneously.

In [ ]:
plp(df, heatmap=True, cluster=True)

`plp` can create a buffer zone around each point. This buffer is a circular area centered on the point, useful for visualizing spatial influence or conducting proximity-based analysis, such as identifying features within 100 meters of a crash site.

In [ ]:
plp(df, buffer_radius=100)

`plp` can also create a ring-shaped buffer, sometimes called a "donut buffer," around each point. Each ring is defined by an inner and outer radius. For example, the code in the next cell creates a ring that starts 100 meters from each point and extends out to 200 meters. This is useful when you want to exclude the immediate area around a point and focus on a specific surrounding zone instead.

In [ ]:
plp(df, ring_inner_radius=100, ring_outer_radius=200)

If no columns contain `lat` and `lon` keywords, or if more than two columns contain these keywords, you must explicitly specify the latitude and longitude using the `y` and `x` parameters, respectively, e.g., `plp(df, x="easting", y="northing")`. Note that `plp` assumes all data is in the [EPSG:4326](https://epsg.io/4326) projection. 

For a `GeoDataFrame`, the `plp` function can render Shapely objects such as `Point`, `LineString`, `Polygon`, and `MultiPolygon`.


Furthermore, GeoParse can visualize geospatial grids (including geohash, S2, and H3) and OSM ways (lines and polygons) directly on the map.


In [ ]:
gdf = gpd.read_file("https://geoparse.io/tutorials/data/london.geojson")
gdf

In [ ]:
plp(gdf)

In [ ]:
SnabbKarta.plp(gdf, centroid=True)

In [ ]:
SnabbKarta.plp(gdf, h3_res=8, compact=True)

In [ ]:
from geoparse import SpatialIndex

In [ ]:
%%time
cells, cnt = SpatialIndex.ppoly_cell(
    gdf, cell_type="h3", res=11, force_full_cover=True, compact=False
)
cnt

In [ ]:
%%time
cells, cnt = SpatialIndex.ppoly_cell(
    gdf, cell_type="h3", res=12, force_full_cover=True, compact=True
)
cnt

In [ ]:
plp(gdf, h3_res=7)

In [ ]:
plp(gdf, h3_res=10, compact=True)

In [ ]:
plp(gdf, s2_res=12)

In [ ]:
plp(gdf, s2_res=14, compact=True)

In [ ]:
plp(gdf, geohash_res=5)

In [ ]:
plp(gdf, geohash_res=7, compact=True)

In [ ]:
plp([df, gdf], h3_res=6)


Furthermore, GeoParse can visualize geospatial grids (including geohash, S2, and H3) and OSM ways (lines and polygons) directly on the map.


In [ ]:
import h3

lat, lon = 41.87, -87.78

# Get the H3 index at resolution 6 for the central point
h3_index = h3.latlng_to_cell(lat, lon, 6)

# Get adjacent H3 cells including the central cell itself
h3_cells = h3.grid_disk(h3_index, 1)  # k_ring with radius 1 returns the central cell + neighbors
h3_cells

In [ ]:
plp(cells=list(h3_cells), cell_type="h3")

In [ ]:
dict1 = {'geom_list': h3_cells,
        'geom_type': 'h3'}

In [ ]:
SnabbKarta.plp(dict1)

In [ ]:
plp(osm_ways=[335265936, 53820456, 1117218957], s2_res=20, compact=True)

In [ ]:
dict2 = {'geom_list': [335265936, 53820456, 1117218957],
      'geom_type': 'osm'}
dict2

In [ ]:
SnabbKarta.plp(dict2,               
               s2_res=22, compact=True
            )

In [ ]:
df = pd.DataFrame({'osm_id': [335265936, 53820456, 1117218957],
                  'name': ['b1', 'b2', 'b3']}
                 )
df

In [ ]:
SnabbKarta.plp(df, 
               geom_col='osm_id', 
               geom_type='osm'
              )

# UPRN

In [ ]:
%%time 
gdf = pd.read_parquet('~/repo/open-data/data/os-open-uprn/osopenuprn_202509.parquet')
gdf = gpd.GeoDataFrame(gdf, geometry=gpd.points_from_xy(gdf.lon, gdf.lat), crs="EPSG:4326")
gdf = gdf.drop(columns=['lat', 'lon'])
gdf.head()

In [ ]:
uprn_list = gdf.sample(1000_000).uprn.to_list()

In [ ]:
u = {'geom_type': 'uprn', 'geom_list': uprn_list}

In [ ]:
SnabbKarta.plp(u, aux_gdf=gdf, aux_geom_id='uprn')

In [ ]:
df = pd.DataFrame(u)
del df['geom_type']
df.head()

In [ ]:
SnabbKarta.plp(df, geom_type='uprn', geom_col='geom_list', aux_gdf=gdf, aux_geom_id='uprn')

In [ ]:
SnabbKarta.plp(udf.sample(100_000))

In [ ]:
udf.columns = ['UPRN', 'LAT', 'LON']

In [ ]:
%%time
SnabbKarta.plp(df.sample(10000), 
               geom_col='uprn', 
               geom_type='uprn',
               
               uprn_df=udf, 
               uprn_col='UPRN',
               lat_col='LAT',
               lon_col='LON',
 )

In [ ]:
uprn_list = udf.sample(10000).UPRN.to_list()

In [ ]:
uprn_dict = {'geom_type':'uprn', 'geom_list':uprn_list}

In [ ]:
SnabbKarta.plp(uprn_dict, aux_gdf=udf, aux_geom_id='UPRN')

In [ ]:
if uprn_df is None:
    uprn_df = pd.read_parquet("https://geoparse.io/tutorials/data/osopenuprn_202507.parquet")
    uprn_col, lat_col, lon_col = uprn_df.columns

gdf = gdf.sort_values(by=geom_col).reset_index(drop=True)
gdf = gdf.merge(uprn_df, left_on=geom_col, right_on=uprn_col, how="left")
gdf = gpd.GeoDataFrame(gdf, geometry=gpd.points_from_xy(gdf[lon_col], gdf[lat_col]), crs="EPSG:4326")
gdf = gdf.drop(columns=[lat_col, lon_col])


In [ ]:
%%time
SnabbKarta.plp(df.sample(10000), 
               geom_col='uprn', 
               geom_type='uprn',
               
               uprn_df=udf, 
               uprn_col='UPRN',
               lat_col='LAT',
               lon_col='LON',
 )

In [ ]:
%%time
SnabbKarta.plp({'geom_type': 'uprn', 'geom_list': df.uprn.values},
               
               # uprn_df=udf, 
               # uprn_col='UPRN',
               # lat_col='LAT',
               # lon_col='LON',
              )

In [ ]:
df = df.sort_values('uprn').reset_index(drop=True)

In [ ]:
%%time
df = df.merge(udf, on='uprn', how='left')
df.head()

In [ ]:
df.basement.value_counts(normalize=True)

In [ ]:
%%time
df = pd.read_csv("../data/uprn/osopenuprn_202507.csv")
df.head()

In [ ]:
df.head()

In [ ]:
df.columns = ['uprn', 'lat', 'lon']

In [ ]:
df.isnull().sum()

In [ ]:
len(df)

In [ ]:
df.uprn.nunique()

In [ ]:
df.to_parquet('../data/uprn/osopenuprn_202507.parquet', index=False)

# USRN

In [ ]:
gdf = gpd.read_parquet('../../open-data/data/os-open-roads/road_link.parquet', bbox=(-0.5,51.3, 0.5,52))
len(gdf)

In [ ]:
gdf.head(2)

In [ ]:
SnabbKarta.plp(gdf.sample(100_000))

In [ ]:
%%time
gdf = gpd.read_parquet('/Users/abbas/repo/open-data/data/os-open-usrn/osopenusrn_202509.parquet', bbox=(-0.5,51.25, 0.5,51.75))
#gdf = gpd.read_file('~/Dropbox/notebook/data/fetch/polygons/uk_rgn.gpkg')
len(gdf)

In [ ]:
u = {'geom_type': 'usrn', 'geom_list': gdf.usrn.to_list()}

In [ ]:
SnabbKarta.plp(u, aux_gdf=gdf, aux_geom_id='usrn')

In [ ]:
df = pd.DataFrame(u)
del df['geom_type']
df.head()

In [ ]:
SnabbKarta.plp(df, geom_type='usrn', geom_col='geom_list', 
               aux_gdf=gdf, aux_geom_id='usrn')

In [ ]:
gdf.head()

In [ ]:
len(gdf)

In [ ]:
import shapely

In [ ]:
type(gdf.geometry.type.unique()[0])

In [ ]:
for g in gdf.geometry.type.unique():
    print(len(gdf[gdf.geometry.type==g]))

In [ ]:
df = pd.read_csv('/Users/abbas/Dropbox/notebook/data/fetch/macrodb/postcode_doogal/sectors_10100.csv.zip')
df['geometry'] = [shapely.wkb.loads(item, hex=True) for item in df.geom]

gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf = gdf[['name', 'geometry']]
gdf.columns = ['pc_sector', 'geometry']
gdf.head()

In [ ]:
SnabbKarta.plp(gdf)

In [ ]:
gdf.geometry.type.value_counts()

In [ ]:
reg_list = ['North West', 'London']

In [ ]:
reg_dict = {'geom_type': 'postcode', 'geom_list': reg_list}

In [ ]:
SnabbKarta.plp(reg_dict, 
               aux_gdf=gdf, aux_geom_id='name'
              )

In [ ]:
pc_list = gdf.sample(1000).pc_sector.to_list()
pc_dict = {'geom_type':'postcode', 'geom_list': pc_list}

In [ ]:
SnabbKarta.plp(pc_dict, 
               aux_gdf=gdf, aux_geom_id='pc_sector'
              )

In [ ]:
usrn_list = gdf.sample(100000).usrn.to_list()

In [ ]:
usrn_dict = {'geom_type':'usrn', 'geom_list': usrn_list}

In [ ]:
SnabbKarta.plp(usrn_dict, 
               aux_gdf=gdf, aux_geom_id='usrn'
              )

In [ ]:
df = pd.DataFrame({'street_id': usrn_list})
df.head()

In [ ]:
SnabbKarta.plp(df, geom_type='usrn', geom_col='street_id',
               aux_gdf=gdf, aux_geom_id='usrn'
              )

In [ ]:
gdf

In [ ]:
%%time
gdf = gpd.read_file('../data/usrn/osopenusrn_202509_processed.gpkg')
len(gdf)

In [ ]:
gdf.head()

In [ ]:
cdf = gdf.copy()

In [ ]:
from geoparse import GeomUtils

In [ ]:
gdf = cdf.copy()
gdf = gdf.sample(10000).reset_index(drop=True)

In [ ]:
%%time
gdf['geometry'] = GeomUtils.flatten_3d(gdf.geometry)
gdf.head()

In [ ]:
%%time
gdf = gdf.to_crs('epsg:4326')
gdf.head()

In [ ]:
gdf.geometry.geom_type.value_counts()

In [ ]:
gdf.street_type.value_counts(normalize=True)

In [ ]:
plp(gdf[gdf.geometry.geom_type == 'LineString'].sample(10000))

In [ ]:
test_flatten_3d_all_types()